In [1]:
import numpy as np
import pandas as pd
import wrangle_products as w
import re
import nltk

In [2]:
products = w.wrangle_products()

Opening local amazon_cycling.csv file


In [3]:
products.head(2)

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating
0,QUXIS Portable High Pressure Foot Air Pump Hea...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/QUXIS-Portable-Activated...,4.0,7635,5.99,15.59,quxis portable high pressure foot air pump hea...,9.6,30540.0
1,Boldfit Gym Shaker for Protein Shake Leakproof...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Boldfit-Typhoon-Shaker-L...,4.1,8052,2.75,2.75,boldfit gym shaker protein shake leakproof sha...,0.0,33013.2


In [4]:
products.tail()

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating
1112,Cread Craft Winter Wear Men's Poly Cotton Ther...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/61t6DNxN3+...,https://www.amazon.in/Winter-Cotton-Thermal-Se...,0.0,0,5.06,17.99,cread craft winter wear men poly cotton therma...,12.93,0.0
1113,Lux Inferno Women's Round Neck Sleeveless Uppe...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/41KdM4GjzZ...,https://www.amazon.in/Lux-Inferno-Womens-Sleev...,0.0,0,5.68,5.68,lux inferno woman round neck sleeveless upper ...,0.00,0.0
1114,"WEGAS Bigbull 26 Inch Mountain Bike 18.5"" Fram...",sports & fitness,Cycling,https://m.media-amazon.com/images/I/81AoVzv-8s...,https://www.amazon.in/WEGAS-26-Mountain-Standa...,4.3,2,171.00,179.99,wegas bigbull 26 inch mountain bike 185 frame ...,8.99,8.6
1115,AN Collection Socks Winter Warmer Thicken Ther...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/31u52aecz3...,https://www.amazon.in/Collection-Winter-Thicke...,0.0,0,3.23,3.59,collection sock winter warmer thicken thermal ...,0.36,0.0
1116,Unisex Stretchable Woollen Sleeveless Thermal ...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/71UXyugLMJ...,https://www.amazon.in/Unisex-STREATCHABLE-Wool...,0.0,0,5.39,7.19,unisex stretchable woollen sleeveless thermal ...,1.80,0.0


In [5]:
products.shape

(1117, 12)

In [6]:
sub_cat_list = ['Tires and Tubes', 'Gloves', 'Helmets', 'Bike Stands',
       'Mountain Bikes', 'Hydration Packs', 'Jerseys', 'Fenders',
       'Cleaners', 'Socks', 'Caps', 'Touring Bikes', 'Bottles and Cages',
       'Vests', 'Road Bikes', 'Bike Racks', 'Shorts']

In [7]:
cat_list = w.get_cat_list(sub_cat_list, extra_words=['bike', 'bikes'])

### To find the product categories: we will loop through the product list, then on each product, we can loop through the cat_list, and check for matches using regex. If we find a match, we will return the product row index number, and category number based on position in the cat_list. We should also account for products having no matches or many category matches

#### lets start by figuring out the regex match formula for one cat and one row

#### regexp = r'.*{cat}.*'
regex doesn't seem to be able to handle dynamic f string style commands so lets build a table with the original category name, the lemmatized version. along with the regexp to find the lemmatized cat name

In [8]:
cat_df = pd.concat([
    pd.Series(sub_cat_list), 
    pd.Series(cat_list)
    ], axis=1).\
    rename(columns={0:'shop_cat', 1:'lemmed_cat'})

In [9]:
cat_df

,shop_cat,lemmed_cat
0,Tires and Tubes,tire tube
1,Gloves,glove
2,Helmets,helmet
3,Bike Stands,stand
4,Mountain Bikes,mountain
5,Hydration Packs,hydration pack
6,Jerseys,jersey
7,Fenders,fender
8,Cleaners,cleaner
9,Socks,sock


In [10]:
regexp = r'.*tire.*|.*tube.*'

In [11]:
regexp = r'.*horn.*|.*portable.*'

In [12]:
re.search(regexp, products.name_preped[0])

<re.Match object; span=(0, 108), match='quxis portable high pressure foot air pump heavy >

### Lets make a list of regular expressions for each category

In [13]:
regex_list = [r'.*tire.*|.*tube.*',
              r'.*glove.*',
              r'.*helmet.*',
              r'.*stand.*',
              r'.*mountain.*',
              r'.*hydration.*',
              r'.*jersey.*',
              r'.*fender.*',
              r'.*cleaner.*',
              r'.*sock.*',
              r'.*cap.*',
              r'.*touring.*',
              r'.*bottle.*|.*cage.*',
              r'.*vest.*',
              r'.*road.*',
              r'.*rack.*',
              r'.*short.*']

In [14]:
cat_df = pd.concat([cat_df, pd.Series(regex_list)], axis=1).rename(columns={0:'regexp'})

In [15]:
cat_df

,shop_cat,lemmed_cat,regexp
0,Tires and Tubes,tire tube,.*tire.*|.*tube.*
1,Gloves,glove,.*glove.*
2,Helmets,helmet,.*helmet.*
3,Bike Stands,stand,.*stand.*
4,Mountain Bikes,mountain,.*mountain.*
5,Hydration Packs,hydration pack,.*hydration.*
6,Jerseys,jersey,.*jersey.*
7,Fenders,fender,.*fender.*
8,Cleaners,cleaner,.*cleaner.*
9,Socks,sock,.*sock.*


### Time to loop. We will loop through the products.name_preped, and on each product, loop through cat_df.regexp to find matches. We will then create a list of products for each category. So we will want the product row# along with the category# for any matches.

In [16]:
for i, product in enumerate(products.name_preped[7:8]):
    print(i)
    print(product)
    for j, cat in enumerate(cat_df.regexp):
        print(j)
        print(cat)
        if re.search(cat, product):
            print('yes')

0
pursue fashion premium cotton ankle sock men woman free size solid unisex ankle length sock pack 5 multi
0
.*tire.*|.*tube.*
1
.*glove.*
2
.*helmet.*
3
.*stand.*
4
.*mountain.*
5
.*hydration.*
6
.*jersey.*
7
.*fender.*
8
.*cleaner.*
9
.*sock.*
yes
10
.*cap.*
11
.*touring.*
12
.*bottle.*|.*cage.*
13
.*vest.*
14
.*road.*
15
.*rack.*
16
.*short.*


In [17]:
cat_matches = {}
for i, product in enumerate(products.name_preped):
    matching_cats = []
#     print(i)
#     print(product)
    for j, cat in enumerate(cat_df.regexp):
#         print(j)
#         print(cat)
        if re.search(cat, product):
            matching_cats.append(j)
#         for k in matching_cats:
        cat_matches[i] = matching_cats
cat_matches

{0: [],
 1: [12],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [9],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [12],
 13: [4],
 14: [],
 15: [],
 16: [],
 17: [12],
 18: [12],
 19: [],
 20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: [4],
 26: [14],
 27: [12],
 28: [9],
 29: [],
 30: [],
 31: [],
 32: [9],
 33: [],
 34: [],
 35: [],
 36: [],
 37: [16],
 38: [2, 4],
 39: [4],
 40: [],
 41: [],
 42: [4],
 43: [],
 44: [],
 45: [9],
 46: [],
 47: [3, 4, 14],
 48: [4],
 49: [9],
 50: [5],
 51: [],
 52: [9],
 53: [12],
 54: [],
 55: [],
 56: [],
 57: [12],
 58: [],
 59: [],
 60: [9],
 61: [12],
 62: [],
 63: [],
 64: [],
 65: [9],
 66: [9],
 67: [],
 68: [],
 69: [],
 70: [],
 71: [12],
 72: [],
 73: [],
 74: [],
 75: [],
 76: [],
 77: [15],
 78: [],
 79: [0, 4, 14],
 80: [],
 81: [9],
 82: [4, 15],
 83: [3, 12, 15],
 84: [],
 85: [],
 86: [],
 87: [],
 88: [9],
 89: [],
 90: [15],
 91: [],
 92: [4],
 93: [9],
 94: [],
 95: [9],
 96: [],
 97: [],
 98: [10],
 99: [],
 100: [14],
 101: [],
 102: 

In [45]:
cat_matches[998][0]

9

In [52]:
for i in range(len(cat_matches)):
    for j in cat_matches[i]:
#         print(cat_df[j])
        print(j)

12
9
12
4
12
12
4
14
12
9
9
16
2
4
4
4
9
3
4
14
4
9
5
9
12
12
9
12
9
9
12
15
0
4
14
9
4
15
3
12
15
9
15
4
9
9
10
14
14
9
4
9
9
10
0
9
3
9
4
15
9
0
3
6
3
4
12
0
12
9
9
9
0
9
9
7
4
2
16
9
16
9
9
14
9
9
6
9
12
9
14
9
9
9
6
9
9
9
9
9
9
6
9
16
9
10
9
9
16
4
14
9
6
3
15
16
9
12
9
9
9
4
15
4
3
4
15
12
6
9
9
3
4
9
3
12
3
14
9
9
9
4
6
9
4
14
0
16
4
14
0
4
13
3
12
15
9
15
9
9
4
14
9
0
16
0
6
16
9
9
9
9
9
9
9
9
9
14
9
13
15
3
9
9
9
0
4
9
9
14
12
12
9
4
9
14
3
9
9
9
9
9
9
3
9
4
14
5
12
9
12
6
4
6
16
9
14
10
9
4
14
15
15
6
0
14
9
9
9
12
9
9
10
12
9
4
9
9
11
14
12
15
9
12
12
9
9
0
4
7
6
4
0
9
16
9
9
9
9
4
14
12
12
3
9
15
9
9
0
9
9
9
9
16
9
12
4
9
9
9
9
9
9
4
14
6
0
4
6
9
9
0
6
9
0
4
9
9
15
9
14
9
9
12
9
10
9
9
9
9
9
10
9
9
0
6
16
9
9
9
2
10
9
4
14
9
15
14
9
16
9
9
14
9
4
14
3
15
10
12
9
14
12
10
9
16
13
4
14
6
9
9
0
2
14
6
9
9
9
9
4
9
9
12
16
4
14
9
9
0
9
0
9
9
9
13
9
9
14
9
9
14
9
6
9
6
9
9
12
12
9
9
0
9
9
4
9
4
2
12
9
4
9
4
4
13
12
15
9
10
14
4
14
9
9
9
13
0
9
9
16
14
4
14
12
4
13
14
2
6
9
9
16
15

In [18]:
len(cat_matches)

1117

In [19]:
pd.concat([products, pd.Series(cat_matches)], axis=1)

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating,0
0,QUXIS Portable High Pressure Foot Air Pump Hea...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/QUXIS-Portable-Activated...,4.0,7635,5.99,15.59,quxis portable high pressure foot air pump hea...,9.60,30540.0,[]
1,Boldfit Gym Shaker for Protein Shake Leakproof...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Boldfit-Typhoon-Shaker-L...,4.1,8052,2.75,2.75,boldfit gym shaker protein shake leakproof sha...,0.00,33013.2,[12]
2,Techista 2-in-1 Rechargeable - Cycle Light (3 ...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Techista-2-1-Rechargeabl...,4.1,2214,4.07,11.99,techista 2in1 rechargeable cycle light 3 mode ...,7.92,9077.4,[]
3,"Nivia Plain Encounter Stockings (L, Black) - P...",sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Nivia-728LBW-Encounter-B...,4.1,2974,1.74,1.79,nivia plain encounter stocking l black polyest...,0.05,12193.4,[]
4,Eazo Steel Multipurpose Air Pump with needle (...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/EAZO-Steel-Multipurpose-...,4.1,3678,4.43,5.40,eazo steel multipurpose air pump needle use ca...,0.97,15079.8,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112,Cread Craft Winter Wear Men's Poly Cotton Ther...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/61t6DNxN3+...,https://www.amazon.in/Winter-Cotton-Thermal-Se...,0.0,0,5.06,17.99,cread craft winter wear men poly cotton therma...,12.93,0.0,[]
1113,Lux Inferno Women's Round Neck Sleeveless Uppe...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/41KdM4GjzZ...,https://www.amazon.in/Lux-Inferno-Womens-Sleev...,0.0,0,5.68,5.68,lux inferno woman round neck sleeveless upper ...,0.00,0.0,[13]
1114,"WEGAS Bigbull 26 Inch Mountain Bike 18.5"" Fram...",sports & fitness,Cycling,https://m.media-amazon.com/images/I/81AoVzv-8s...,https://www.amazon.in/WEGAS-26-Mountain-Standa...,4.3,2,171.00,179.99,wegas bigbull 26 inch mountain bike 185 frame ...,8.99,8.6,"[3, 4]"
1115,AN Collection Socks Winter Warmer Thicken Ther...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/31u52aecz3...,https://www.amazon.in/Collection-Winter-Thicke...,0.0,0,3.23,3.59,collection sock winter warmer thicken thermal ...,0.36,0.0,[9]


## Switching gears. We are going to look at recommending products based on avg cluster spending amount, and product review scores, then work on getting category matches later

In [20]:
products.describe()

,ratings,no_of_ratings,discount_price,actual_price,discount_amount,prod_rating
count,1117.000000,1117.000000,1117.000000,1117.000000,1117.000000,1117.000000
mean,2.774575,227.228290,19.441128,31.711889,12.270761,944.593912
std,1.798072,1322.349807,44.937416,69.498948,37.147052,5690.028019
min,0.000000,0.000000,0.710000,1.260000,0.000000,0.000000
25%,0.000000,0.000000,3.660000,7.190000,1.760000,0.000000
50%,3.600000,4.000000,5.990000,11.990000,4.800000,17.200000
75%,4.000000,29.000000,11.770000,19.190000,8.990000,108.800000
max,5.000000,19181.000000,695.880000,719.990000,558.600000,86314.500000


#### Over 75% of the product dataset is priced less than $20

- cluster 0 average revenue is: 179.51208083174697
- cluster 1 average revenue is: 321.6032662487628
- cluster 2 average revenue is: 898.5687723480333
- cluster 3 average revenue is: 2276.8223954060704

### Lets see if we can group products into
- less than 20 dollars
- 20 to 40 dollars
- 40 to 100 dollars
- above 100 dollars

In [21]:
len(products[(products.actual_price <20)])

848

In [22]:
len(products[(products.actual_price >= 20) & (products.actual_price < 40)])

111

In [23]:
len(products[(products.actual_price >= 40) & (products.actual_price < 100)])

69

In [24]:
len(products[(products.actual_price >= 100)])

89

In [25]:
group_1 = products[(products.actual_price <20)]
group_2 = products[(products.actual_price >= 20) & (products.actual_price < 40)]
group_3 = products[(products.actual_price >= 40) & (products.actual_price < 100)]
group_4 = products[(products.actual_price >= 100)]

In [26]:
group_1 = group_1.sort_values('prod_rating', ascending=False).\
        reset_index().drop(columns='index')
group_2 = group_2.sort_values('prod_rating', ascending=False).\
        reset_index().drop(columns='index')
group_3 = group_3.sort_values('prod_rating', ascending=False).\
        reset_index().drop(columns='index')
group_4 = group_4.sort_values('prod_rating', ascending=False).\
        reset_index().drop(columns='index')

In [27]:
group_3.head()

,name,amazon_main_cat,amazon_sub_cat,image,link,ratings,no_of_ratings,discount_price,actual_price,name_preped,discount_amount,prod_rating
0,Luusa TFT RX-500 Plug N Play Kids/Baby Tricycl...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Luusa-RX-500-Tricycle-Pa...,4.1,12842,28.19,44.39,luusa tft rx500 plug n play kid baby tricycle ...,16.20,52652.2
1,Nike Men's Calf Socks (Pack of 3) (SX4700-901_...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Performance-Cushion-Trai...,4.6,4200,47.40,47.40,nike men calf sock pack 3 sx4700901grey black ...,0.00,19320.0
2,Leader Scout MTB 26T Mountain Bicycle/Bike Wit...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Leader-Scout-26T-SEA-Gre...,4.1,3457,53.99,90.00,leader scout mtb 26t mountain bicycle bike wit...,36.01,14173.7
3,MICHELIN 12206 Analogue Double Barrel Foot Pum...,sports & fitness,Cycling,https://m.media-amazon.com/images/I/51mDMyokaC...,https://www.amazon.in/Michelin-12206-Analogue-...,4.3,2589,25.14,46.02,michelin 12206 analogue double barrel foot pum...,20.88,11132.7
4,R for Rabbit Tiny Toes Grand Kids Baby Tricycl...,sports & fitness,Cycling,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Rabbit-Tiny-Toes-Grand-C...,4.3,2472,71.93,95.90,r rabbit tiny toe grand kid baby tricycle pare...,23.97,10629.6


## Most common purchase sub_category by customer cluster
Cluster 0
- Tires and Tubes      3410
- Bottles and Cages    1762
- Caps                  485
- Helmets               331
- Cleaners              172

Cluster 1
- Tires and Tubes      2545
- Bottles and Cages    1207
- Helmets               706
- Caps                  337
- Jerseys               277

Cluster 2
- Helmets            1313
- Road Bikes          900
- Jerseys             533
- Tires and Tubes     328
- Mountain Bikes      297

Cluster 3
- Mountain Bikes     1133
- Road Bikes          543
- Touring Bikes       388
- Jerseys             131
- Shorts              105